In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

%matplotlib inline

In [2]:
#Read in data - Unsure on age categories - so have just named them Age1 through Age7
df=pd.read_csv("wu02uk_msoa_v2.csv", header=None)
df=df.rename(columns={0:"From", 1:"To", 2:"Age1", 3:"Age2",
                  4:"Age3", 5:"Age4", 6:"Age5", 7:"Age6", 8:"Age7"})
df.head()

,From,To,Age1,Age2,Age3,Age4,Age5,Age6,Age7
0,95AA01S1,95AA01S1,15,1,6,7,1,0,0
1,95AA01S1,95AA01S2,207,42,105,60,0,0,0
2,95AA01S1,95AA01S3,3,0,2,1,0,0,0
3,95AA01S1,95AA02W1,1,0,0,1,0,0,0
4,95AA01S1,95AA04W1,5,1,2,2,0,0,0


## Shapefile merging, read from internet sources. Areas not included in flow data

Outputs geom - full geometry shapefile for total UK countries. Length > dataset - unsure how you want shapefiles coded that aren't in the flow dataset - so have left them enumerated at the bottom, but they just don't have a mapping in the flows data.

In [3]:
EWMSOA=gpd.read_file('Middle_Layer_Super_Output_Areas_December_2011_Boundaries_EW_BFC')

In [5]:
SIZ=gpd.read_file('http://sedsh127.sedsh.gov.uk/Atom_data/ScotGov/ZippedShapefiles/SG_IntermediateZoneBdry_2011.zip')

In [6]:
#NISOA=gpd.read_file('https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/SOA2011_Esri_Shapefile_0.zip')
NISOA=gpd.read_file('SOA2011_Esri_Shapefile_0')

In [7]:
EWMSOA=EWMSOA.to_crs("EPSG:4326") # to make shapefiles consistent and in lat long
NISOA=NISOA.to_crs("EPSG:4326") # google earth stuff.
SIZ=SIZ.to_crs("EPSG:4326")

In [8]:
EWMSOA.rename(columns={"msoa11cd":"cd"}, inplace=True)
NISOA.rename(columns={"SOA_CODE":"cd"}, inplace=True)
SIZ.rename(columns={"InterZone":"cd"}, inplace=True)

In [9]:
geom=gpd.GeoDataFrame(EWMSOA[{"cd","geometry"}])
geom=geom.append(NISOA[{"cd","geometry"}], ignore_index=True)
geom=geom.append(SIZ[{"cd","geometry"}], ignore_index=True)
geom.shape

(9370, 2)

## Create Master lookup for True IDs (TID) including empty-cells and non UK destinations

In [26]:
df9=df[['From','To']]
df9.head()
column_values = df9[["From", "To"]].values.ravel() # turns them into an array from a series
unique_values =  pd.unique(column_values)



lookup=pd.DataFrame({"CD":list(df.To.unique())}) # make lookup data frame with CD column as unique indicator
lookup=lookup.merge(geom, left_on="CD", right_on="cd", how="outer") # add ont he geoms from the shapefiles
lookup.cd.fillna(lookup.CD, inplace=True) # fill in NA where you don't have geoms


In [30]:
len(unique_values)

9330

In [28]:
lookup.shape

lookup['TID']=np.arange(len(lookup))+1



(10609, 4)

In [29]:
lookupdict=dict(zip(lookup.cd, lookup.TID))
df.From=df.From.map(lookupdict)
df.To=df.To.map(lookupdict)

In [31]:
df.to_csv('ageflows.csv',index=False)

In [32]:
df.head()

header = ["From","To","Age1"]
df.to_csv('output.csv', columns = header,index=False)

In [27]:
df.head()

,From,To,Age1,Age2,Age3,Age4,Age5,Age6,Age7,Total
0,1,1,15,1,6,7,1,0,0,30
1,1,2,207,42,105,60,0,0,0,414
2,1,3,3,0,2,1,0,0,0,6
3,1,4,1,0,0,1,0,0,0,2
4,1,5,5,1,2,2,0,0,0,10


In [33]:
geom['ID']=geom.cd.map(lookupdict)
geom=geom.sort_values(by=['ID'])
geom.to_file("fullgeom.shp")

## Generating Work and Play Files

In [35]:
#work=pd.DataFrame(df[{"From", "Age1", "Age2", "Age3", "Age4", 
#                      "Age5", "Age6", "Age7"}].groupby('From').sum())

work=pd.DataFrame(df[{"From", "Age1"}].groupby('From').sum())

#work['sum']=work.loc[:,work.columns != "From"].sum(axis=1) # calculating the sum over columns Age 2 onwards
work.reset_index(inplace=True)
work=work[['From', 'Age1']]
work.to_csv('worksize.csv',index=False)

## Generating Play file - take MSOA, IZ, SOA population estimates from gov sources and compile into master list.
 Merge with work file and subtract work population flow from the total population to get estimate of "play". 
 
 Note 571 areas have "from" flows recorded which are higher than the estimated population - meaning negative estimates
 for play population.

In [36]:
#Taking Scottish Data and converting to Two Column DataFrame with Code (cd) and Total Population (tot)
scotpop=pd.read_csv("iz2011-pop-est_02042020.csv", skiprows=3, header=None)



scotpop1=scotpop[scotpop[0]==2018]
scotpop1=scotpop1[{1,3,4}]
scotpop1=scotpop1.iloc[2:,]
scotpop1=scotpop1.groupby(1).sum()
scotpop1=scotpop1.reset_index()
scotpop1=scotpop1.rename(columns={1:"cd", 4:"tot"})

In [37]:
#Taking Northern Irish Data and converting to Two Column DataFrame with Code (cd) and Total Population (tot)
nipop=pd.read_csv("super-output-areas-soas-by-gender-and-broad-age-bands-mid-2001-to-mid-2018.csv")
nipop1=nipop[nipop['Age_Group']=="All ages"]
nipop1=nipop1[nipop1['Mid_Year_Ending']==2018]
nipop1=nipop1[nipop1['Gender']=="All persons"]
nipop1=nipop1[{"Geo_Code", "Population_Estimate"}]
nipop1=nipop1.rename(columns={"Geo_Code":"cd", "Population_Estimate":"tot"})

In [38]:
## File from https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fmiddlesuperoutputareamidyearpopulationestimatesnationalstatistics%2fmid2018sape21dt14a/sape21dt15mid2018msoaquinaryestimatesunformatted.zip
#Taking England and Wales Data and converting to Two Column DataFrame with Code (cd) and Total Population (tot)
# Not read directly from link as indexing subsheets in zipped xlsx format is lengthy. Can be updated
ewpop=pd.read_excel("SAPE21DT15-mid-2018-msoa-quinary-estimates-unformatted.xlsx",
                    sheet_name="Mid-2018 Persons", 
                    skiprows=4)
ewpop=ewpop[{"Area Codes", "All Ages"}]
ewpop=ewpop.rename(columns={"Area Codes":"cd", "All Ages":"tot"})

In [39]:
# Generate master population estimate DataFrame (cd, tot)
totpop=ewpop.append(nipop1, ignore_index=True).append(scotpop1, ignore_index=True)
totpop['ID']=totpop.cd.map(lookupdict)
totpop.sort_values(by=['tot'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,cd,tot,ID
8065,95YY15S1,382,673
7679,95MM14S1,881,1889
8780,S02001925,885,10020
7201,95AA01S1,1026,1
7795,95QQ03W1,1038,1744
...,...,...,...
4330,E02000891,19374,331
243,E02001731,19855,929
242,E02001730,19868,1758
5103,E02003475,21716,2759


#### Finally generate playsize.csv by merging total population file with work file and generating "play" column

In [41]:
#Merge totpop with work file and use lookupdict to convert cd into master list of areas.  
#newpop=totpop.merge(lookup[{"CD", "TID"}], left_on="cd", right_on="CD", how="left")

newpop=totpop.merge(work, left_on="ID", right_on="From", how="left")


In [46]:
newpop['Age1'].replace(np.nan, 0, inplace=True)
newpop['play']=newpop['tot'] - newpop['Age1']

newpop.dtypes

cd       object
tot       int64
ID        int64
From    float64
Age1    float64
play    float64
dtype: object

In [50]:


playpop=newpop[['ID', 'play']]
playpop=playpop.sort_values(by=['ID'])

playpop.astype('int64')

playpop.dtypes
#playpop.to_csv('playsize.csv',index=False)

ID        int64
play    float64
dtype: object

In [22]:
playpop[playpop['play']<=0].shape

(1718, 2)

In [23]:
playpop.head

<bound method NDFrame.head of          ID    play
7201      1  -156.0
7202      2   232.0
7203      3   -27.0
7204      4   106.0
7206      5   155.0
...     ...     ...
9365  10605  5489.0
9366  10606  4363.0
9367  10607  5631.0
9368  10608  3866.0
9369  10609  2889.0

[9370 rows x 2 columns]>